In [242]:
import numpy as np 
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
%matplotlib inline
pd.set_option('display.max_columns', 100)

In [326]:
df = pd.read_csv('cleaned_df', index_col = 0)


In [328]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,year_most_recent_construction,years_since_most_recent_construction,yard,excess_bathrooms,zip_total_households,zip_median_income,zip_mean_income,zip_total_families,zip_total_nonfamilies,season_sold,log_price,log_sqft_living,excess_bathrooms_cat,view_cat,10yr_ysmrc,decade_built,condition_adjusted,renovated_after_2005,decade_built_adjusted,grade_cat
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014,10,1986,28,7858.0,-1.0,15830,92225,107395,11422,4408,Fall,12.807653,7.635304,1_less,no_view,30.0,1980s,4,0,1980s,8
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014,8,1977,37,2760.0,-2.0,22922,111569,147430,12426,10496,Summer,13.670485,7.972466,2_or_less,no_view,40.0,1970s,5,0,1970s,8
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014,8,1997,17,9008.0,-1.0,13337,144247,186548,10531,2806,Summer,13.852806,8.234830,1_less,ok_view,20.0,1990s,3,0,1990s,11
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015,3,1990,25,12328.0,1.0,17646,101674,128063,11334,6312,Spring,14.214287,8.425078,1_or_more,ok_view,20.0,1990s,3,0,1990s,12
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014,7,2004,10,4101.0,-0.0,28022,135287,161369,18806,9216,Summer,13.474428,7.843849,parity,no_view,10.0,2000s,3,0,2000s,9


In [329]:
df_dummy = pd.get_dummies(df, columns=['season_sold', 'condition', 'condition_adjusted'], drop_first=True)
df_dummy = pd.get_dummies(df_dummy, columns = ['grade_cat']).drop(columns = 'grade_cat_low_grade', axis = 1)
df_dummy = pd.get_dummies(df_dummy, columns = ['view_cat']).drop(columns = 'view_cat_no_view', axis = 1)
df_dummy = pd.get_dummies(df_dummy, columns = ['excess_bathrooms_cat']).drop(columns = 'excess_bathrooms_cat_parity', axis = 1)
df_dummy = pd.get_dummies(df_dummy, columns = ['excess_bathrooms']).drop(columns = 'excess_bathrooms_-0.0', axis = 1)
df_dummy = pd.get_dummies(df_dummy, columns = ['decade_built']).drop(columns = 'decade_built_2010s', axis = 1)
df = pd.get_dummies(df_dummy, columns = ['decade_built_adjusted']).drop(columns = 'decade_built_adjusted_2010s', axis = 1)

In [244]:
sm.add_constant(df)

,const,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,year_most_recent_construction,years_since_most_recent_construction,yard,zip_total_households,zip_median_income,zip_mean_income,zip_total_families,zip_total_nonfamilies,log_price,log_sqft_living,10yr_ysmrc,renovated_after_2005,season_sold_Spring,season_sold_Summer,season_sold_Winter,condition_2,condition_3,condition_4,condition_5,condition_adjusted_3,condition_adjusted_4,condition_adjusted_5,grade_cat_Low_Grade,grade_cat_Mid_Grade,view_cat_great_view,view_cat_ok_view,excess_bathrooms_cat_1_less,excess_bathrooms_cat_1_or_more,excess_bathrooms_cat_2_or_less,excess_bathrooms_-4.0,excess_bathrooms_-3.0,excess_bathrooms_-2.0,excess_bathrooms_-1.0,excess_bathrooms_1.0,excess_bathrooms_2.0,decade_built_1900s,decade_built_1910s,decade_built_1920s,decade_built_1930s,decade_built_1940s,decade_built_1950s,decade_built_1960s,decade_built_1970s,decade_built_1980s,decade_built_1990s,decade_built_2000s,decade_built_adjusted_1940s,decade_built_adjusted_1950s,decade_built_adjusted_1960s,decade_built_adjusted_1970s,decade_built_adjusted_1980s,decade_built_adjusted_1990s,decade_built_adjusted_2000s,decade_built_adjusted_Pre-war
0,1.0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014,10,1986,28,7858.000000,15830,92225,107395,11422,4408,12.807653,7.635304,30.0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,1.0,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014,8,1977,37,2760.000000,22922,111569,147430,12426,10496,13.670485,7.972466,40.0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,1.0,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014,8,1997,17,9008.000000,13337,144247,186548,10531,2806,13.852806,8.234830,20.0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,1.0,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015,3,1990,25,12328.000000,17646,101674,128063,11334,6312,14.214287,8.425078,20.0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,1.0,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014,7,2004,10,4101.000000,28022,135287,161369,18806,9216,13.474428,7.843849,10.0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17284,1.0,627300195,20150303T000000,750000.0,5,2.50,3240,9960,1.0,0,1,8,2020,1220,1958,0,98008,47.5858,-122.112,2730,10400,2015,3,1958,57,5500.000000,9278,117775,157380,6934,2344,13.527828,8.083329,60.0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
17285,1.0,8819900270,20140520T000000,440000.0,2,1.75,1300,4000,2.0,0,0,7,1300,0,1948,0,98105,47.6687,-122.288,1350,4013,2014,5,1948,66,3350.000000,17146,62582,118967,6322,10824,12.994530,7.170120,70.0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
17286,1.0,3816300095,20140514T000000,310000.0,3,1.00,1050,9876,1.0,0,0,7,1050,0,1953,0,98028,47.7635,-122.262,1760,9403,2014,5,1953,61,8826.000000,8876,110058,135105,6072,2804,12.644328,6.956545,60.0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
17287,1.0,122069107,20141204T000000,427500.0,3,1.50,1900,43186,1.5,0,0,7,1300,

In [228]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,year_most_recent_construction,years_since_most_recent_construction,yard,zip_total_households,zip_median_income,zip_mean_income,zip_total_families,zip_total_nonfamilies,log_price,log_sqft_living,10yr_ysmrc,renovated_after_2005,season_sold_Spring,season_sold_Summer,season_sold_Winter,condition_2,condition_3,condition_4,condition_5,condition_adjusted_3,condition_adjusted_4,condition_adjusted_5,grade_cat_Low_Grade,grade_cat_Mid_Grade,view_cat_great_view,view_cat_ok_view,excess_bathrooms_cat_1_less,excess_bathrooms_cat_1_or_more,excess_bathrooms_cat_2_or_less,excess_bathrooms_-4.0,excess_bathrooms_-3.0,excess_bathrooms_-2.0,excess_bathrooms_-1.0,excess_bathrooms_1.0,excess_bathrooms_2.0,decade_built_1900s,decade_built_1910s,decade_built_1920s,decade_built_1930s,decade_built_1940s,decade_built_1950s,decade_built_1960s,decade_built_1970s,decade_built_1980s,decade_built_1990s,decade_built_2000s,decade_built_adjusted_1940s,decade_built_adjusted_1950s,decade_built_adjusted_1960s,decade_built_adjusted_1970s,decade_built_adjusted_1980s,decade_built_adjusted_1990s,decade_built_adjusted_2000s,decade_built_adjusted_Pre-war,const
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014,10,1986,28,7858.0,15830,92225,107395,11422,4408,12.807653,7.635304,30.0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014,8,1977,37,2760.0,22922,111569,147430,12426,10496,13.670485,7.972466,40.0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014,8,1997,17,9008.0,13337,144247,186548,10531,2806,13.852806,8.234830,20.0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015,3,1990,25,12328.0,17646,101674,128063,11334,6312,14.214287,8.425078,20.0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014,7,2004,10,4101.0,28022,135287,161369,18806,9216,13.474428,7.843849,10.0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1


In [330]:
columns = list(df.columns)
columns

['id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'year',
 'month',
 'year_most_recent_construction',
 'years_since_most_recent_construction',
 'yard',
 'zip_total_households',
 'zip_median_income',
 'zip_mean_income',
 'zip_total_families',
 'zip_total_nonfamilies',
 'log_price',
 'log_sqft_living',
 '10yr_ysmrc',
 'renovated_after_2005',
 'season_sold_Spring',
 'season_sold_Summer',
 'season_sold_Winter',
 'condition_2',
 'condition_3',
 'condition_4',
 'condition_5',
 'condition_adjusted_3',
 'condition_adjusted_4',
 'condition_adjusted_5',
 'grade_cat_10',
 'grade_cat_11',
 'grade_cat_12',
 'grade_cat_13',
 'grade_cat_6',
 'grade_cat_7',
 'grade_cat_8',
 'grade_cat_9',
 'view_cat_great_view',
 'view_cat_ok_view',
 'excess_bathrooms_cat_1_less',
 'excess_bathrooms_cat_1_or_more',
 'e

In [272]:
reg1_c = [
 'sqft_living',
 'waterfront',
 'zip_mean_income',
 'season_sold_Spring',
 'season_sold_Summer',
 'season_sold_Winter',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
'decade_built_1900s',
 'decade_built_1910s',
 'decade_built_1920s',
 'decade_built_1930s',
 'decade_built_1940s',
 'decade_built_1950s',
 'decade_built_1960s',
 'decade_built_1970s',
 'decade_built_1980s',
 'decade_built_1990s',
 'decade_built_2000s',
 'condition_2',
 'condition_3',
 'condition_4',
 'condition_5',
 'grade',
'renovated_after_2005']

In [273]:
X = sm.add_constant(df.loc[:,reg1_c])
y = df['price']

In [274]:
model = sm.OLS(endog=y, exog=X, hasconst = True).fit()

In [275]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.728
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     1645.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:55:09   Log-Likelihood:            -2.3367e+05
No. Observations:               17249   AIC:                         4.674e+05
Df Residuals:                   17220   BIC:                         4.676e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              -1e+06   4.12e+04    -24.286      0.000   -1.08e+06   -9.19e+05
sqft_living                      143.5803      2.499     57.459      0.000     138.682     148.478
waterfront                      4.796e+05   1.99e+04     24.131      0.000    4.41e+05    5.19e+05
zip_mean_income                    2.5762      0.040     64.033      0.000       2.497       2.655
season_sold_Spring              2.914e+04   3887.506      7.497      0.000    2.15e+04    3.68e+04
season_sold_Summer              5287.1182   3914.671      1.351      0.177   -2386.036     1.3e+04
season_sold_Winter              8950.5470   4496.691      1.990      0.047     136.575    1.78e+04
excess_bathrooms_cat_1_or_more  7.817e+04   7627.259     10.249      0.000    6.32e+04    9.31e+04
excess_bathrooms_cat_2_or_less -3.967e+04   4564.772     -8.691      0.000   -4.86e+04   -3.07e+04
excess_bathrooms_cat_1_less    -2.935e+04   3607.723     -8.135      0.000   -3.64e+04   -2.23e+04
view_cat_great_view             3.204e+05   1.45e+04     22.074      0.000    2.92e+05    3.49e+05
view_cat_ok_view                9.406e+04   5356.456     17.560      0.000    8.36e+04    1.05e+05
decade_built_1900s              2.361e+05   1.06e+04     22.287      0.000    2.15e+05    2.57e+05
decade_built_1910s              2.044e+05      1e+04     20.430      0.000    1.85e+05    2.24e+05
decade_built_1920s              2.069e+05   9026.847     22.917      0.000    1.89e+05    2.25e+05
decade_built_1930s              1.813e+05   1.12e+04     16.181      0.000    1.59e+05    2.03e+05
decade_built_1940s              1.616e+05   8455.795     19.116      0.000    1.45e+05    1.78e+05
decade_built_1950s              1.023e+05   7912.951     12.933      0.000    8.68e+04    1.18e+05
decade_built_1960s              3.844e+04   7768.105      4.948      0.000    2.32e+04    5.37e+04
decade_built_1970s             -2.204e+04   7747.069     -2.845      0.004   -3.72e+04   -6856.986
decade_built_1980s             -4.115e+04   7464.232     -5.513      0.000   -5.58e+04   -2.65e+04
decade_built_1990s             -6.394e+04   7372.822     -8.672      0.000   -7.84e+04   -4.95e+04
decade_built_2000s             -3.405e+04   6902.683     -4.933      0.000   -4.76e+04   -2.05e+04
condition_2                     3.533e+04   4.18e+04      0.846      0.397   -4.65e+04    1.17e+05
condition_3                     3.886e+04   3.88e+04      1.002      0.317   -3.72e+04    1.15e+05
condition_4                     5.698e+04   3.88e+04      1.468      0.142   -1.91e+04    1.33e+05
condition_5                     9.918e+04    3.9e+04      2.542      0.011    2.27e+04    1.76e+05
grade                           1.036e+05   2123.553     48.790      0.000    9.94e+04    1.08e+05
renovated_afte

---

In [276]:
X = sm.add_constant(df.loc[:,reg1_c])
y = df['log_price']

In [277]:
model = sm.OLS(endog=y, exog=X).fit()

In [278]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     2009.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:55:15   Log-Likelihood:                -808.34
No. Observations:               17249   AIC:                             1675.
Df Residuals:                   17220   BIC:                             1900.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             10.4067      0.056    184.342      0.000      10.296      10.517
sqft_living                        0.0002   3.43e-06     53.307      0.000       0.000       0.000
waterfront                         0.3496      0.027     12.832      0.000       0.296       0.403
zip_mean_income                 4.982e-06   5.52e-08     90.330      0.000    4.87e-06    5.09e-06
season_sold_Spring                 0.0491      0.005      9.220      0.000       0.039       0.060
season_sold_Summer                 0.0105      0.005      1.957      0.050   -1.42e-05       0.021
season_sold_Winter                 0.0074      0.006      1.196      0.232      -0.005       0.019
excess_bathrooms_cat_1_or_more     0.0463      0.010      4.430      0.000       0.026       0.067
excess_bathrooms_cat_2_or_less    -0.0615      0.006     -9.825      0.000      -0.074      -0.049
excess_bathrooms_cat_1_less       -0.0382      0.005     -7.718      0.000      -0.048      -0.028
view_cat_great_view                0.3016      0.020     15.158      0.000       0.263       0.341
view_cat_ok_view                   0.1510      0.007     20.567      0.000       0.137       0.165
decade_built_1900s                 0.3234      0.015     22.268      0.000       0.295       0.352
decade_built_1910s                 0.2659      0.014     19.391      0.000       0.239       0.293
decade_built_1920s                 0.2935      0.012     23.721      0.000       0.269       0.318
decade_built_1930s                 0.1991      0.015     12.961      0.000       0.169       0.229
decade_built_1940s                 0.1591      0.012     13.726      0.000       0.136       0.182
decade_built_1950s                 0.0857      0.011      7.896      0.000       0.064       0.107
decade_built_1960s                -0.0206      0.011     -1.936      0.053      -0.041       0.000
decade_built_1970s                -0.0724      0.011     -6.819      0.000      -0.093      -0.052
decade_built_1980s                -0.1010      0.010     -9.868      0.000      -0.121      -0.081
decade_built_1990s                -0.1307      0.010    -12.934      0.000      -0.151      -0.111
decade_built_2000s                -0.0648      0.009     -6.843      0.000      -0.083      -0.046
condition_2                        0.1050      0.057      1.835      0.067      -0.007       0.217
condition_3                        0.2850      0.053      5.357      0.000       0.181       0.389
condition_4                        0.3152      0.053      5.924      0.000       0.211       0.420
condition_5                        0.3909      0.053      7.309      0.000       0.286       0.496
grade                              0.1647      0.003     56.562      0.000       0.159       0.170
renovated_afte

---

In [279]:
#taking out season sold and waterfront, changing condition, to condition adjusted
reg2_c =[
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
    'decade_built_1900s',
 'decade_built_1910s',
 'decade_built_1920s',
 'decade_built_1930s',
 'decade_built_1940s',
 'decade_built_1950s',
 'decade_built_1960s',
 'decade_built_1970s',
 'decade_built_1980s',
 'decade_built_1990s',
 'decade_built_2000s',
 'grade',
'renovated_after_2005']

In [280]:
X = sm.add_constant(df.loc[:,reg2_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     2170.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:56:04   Log-Likelihood:            -2.3404e+05
No. Observations:               17249   AIC:                         4.681e+05
Df Residuals:                   17228   BIC:                         4.683e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           -9.44e+05   1.55e+04    -61.079      0.000   -9.74e+05   -9.14e+05
sqft_living                      145.7983      2.544     57.309      0.000     140.812     150.785
zip_mean_income                    2.5757      0.041     62.746      0.000       2.495       2.656
excess_bathrooms_cat_1_or_more  8.255e+04   7788.899     10.599      0.000    6.73e+04    9.78e+04
excess_bathrooms_cat_2_or_less -4.501e+04   4655.894     -9.668      0.000   -5.41e+04   -3.59e+04
excess_bathrooms_cat_1_less     -3.25e+04   3682.738     -8.825      0.000   -3.97e+04   -2.53e+04
view_cat_great_view             5.234e+05   1.21e+04     43.247      0.000       5e+05    5.47e+05
view_cat_ok_view                1.018e+05   5462.347     18.633      0.000    9.11e+04    1.12e+05
decade_built_1900s               2.56e+05   1.06e+04     24.107      0.000    2.35e+05    2.77e+05
decade_built_1910s              2.289e+05   9971.658     22.951      0.000    2.09e+05    2.48e+05
decade_built_1920s              2.278e+05   8968.486     25.397      0.000     2.1e+05    2.45e+05
decade_built_1930s              2.007e+05   1.13e+04     17.767      0.000    1.79e+05    2.23e+05
decade_built_1940s              1.791e+05   8450.597     21.189      0.000    1.62e+05    1.96e+05
decade_built_1950s              1.196e+05   7873.767     15.191      0.000    1.04e+05    1.35e+05
decade_built_1960s              5.596e+04   7668.478      7.297      0.000    4.09e+04     7.1e+04
decade_built_1970s             -6975.1515   7695.416     -0.906      0.365   -2.21e+04    8108.647
decade_built_1980s             -3.352e+04   7532.831     -4.450      0.000   -4.83e+04   -1.88e+04
decade_built_1990s             -6.108e+04   7519.520     -8.122      0.000   -7.58e+04   -4.63e+04
decade_built_2000s             -3.398e+04   7050.643     -4.819      0.000   -4.78e+04   -2.02e+04
grade                           1.024e+05   2163.677     47.337      0.000    9.82e+04    1.07e+05
renovated_after_2005            4.985e+04    1.2e+04      4.146      0.000    2.63e+04    7.34e+04
==============================================================================
Omnibus:                    11578.294   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           432031.789
Skew:                           2.722   Prob(JB):                         0.00
Kurtosis:                      26.906   Cond. No.                     2.34e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.34e+06. This might indicate that there are
strong multicollinearity or other numerical proble

In [128]:
1.054e+05

105400.0

---

In [268]:
#taking out decade built
reg3_c = [
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
 'grade_5',
 'grade_6',
 'grade_7',
 'grade_8',
 'grade_9',
 'grade_10',
 'grade_11',
 'grade_12',
 'grade_13',
 'condition_adjusted_3',
 'condition_adjusted_4',
 'condition_adjusted_5']

In [269]:
X = sm.add_constant(df.loc[:,reg3_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

---

In [96]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,year_most_recent_construction,years_since_most_recent_construction,yard,excess_bathrooms,zip_total_households,zip_median_income,zip_mean_income,zip_total_families,zip_total_nonfamilies,log_price,log_sqft_living,10yr_ysmrc,season_sold_Spring,season_sold_Summer,season_sold_Winter,decade_built_1910s,decade_built_1920s,decade_built_1930s,decade_built_1940s,decade_built_1950s,decade_built_1960s,decade_built_1970s,decade_built_1980s,decade_built_1990s,decade_built_2000s,decade_built_2010s,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,condition_2,condition_3,condition_4,condition_5,condition_adjusted_3,condition_adjusted_4,condition_adjusted_5,view_cat_great_view,view_cat_ok_view,excess_bathrooms_cat_1_less,excess_bathrooms_cat_1_or_more,excess_bathrooms_cat_2_or_less
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014,10,1986,28,7858.0,-1.0,15830,92225,107395,11422,4408,12.807653,7.635304,30.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014,8,1977,37,2760.0,-2.0,22922,111569,147430,12426,10496,13.670485,7.972466,40.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014,8,1997,17,9008.0,-1.0,13337,144247,186548,10531,2806,13.852806,8.234830,20.0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015,3,1990,25,12328.0,1.0,17646,101674,128063,11334,6312,14.214287,8.425078,20.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014,7,2004,10,4101.0,-0.0,28022,135287,161369,18806,9216,13.474428,7.843849,10.0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0


In [266]:
reg4_c =[
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
 'decade_built_1900s',
 'decade_built_1910s',
 'decade_built_1920s',
 'decade_built_1930s',
 'decade_built_1940s',
 'decade_built_1950s',
 'decade_built_1960s',
 'decade_built_1970s',
 'decade_built_1980s',
 'decade_built_1990s',
 'decade_built_2000s',
 'grade',
 'renovated_after_2005']

In [267]:
X = sm.add_constant(df.loc[:,reg4_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     2170.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:51:07   Log-Likelihood:            -2.3404e+05
No. Observations:               17249   AIC:                         4.681e+05
Df Residuals:                   17228   BIC:                         4.683e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           -9.44e+05   1.55e+04    -61.079      0.000   -9.74e+05   -9.14e+05
sqft_living                      145.7983      2.544     57.309      0.000     140.812     150.785
zip_mean_income                    2.5757      0.041     62.746      0.000       2.495       2.656
excess_bathrooms_cat_1_or_more  8.255e+04   7788.899     10.599      0.000    6.73e+04    9.78e+04
excess_bathrooms_cat_2_or_less -4.501e+04   4655.894     -9.668      0.000   -5.41e+04   -3.59e+04
excess_bathrooms_cat_1_less     -3.25e+04   3682.738     -8.825      0.000   -3.97e+04   -2.53e+04
view_cat_great_view             5.234e+05   1.21e+04     43.247      0.000       5e+05    5.47e+05
view_cat_ok_view                1.018e+05   5462.347     18.633      0.000    9.11e+04    1.12e+05
decade_built_1900s               2.56e+05   1.06e+04     24.107      0.000    2.35e+05    2.77e+05
decade_built_1910s              2.289e+05   9971.658     22.951      0.000    2.09e+05    2.48e+05
decade_built_1920s              2.278e+05   8968.486     25.397      0.000     2.1e+05    2.45e+05
decade_built_1930s              2.007e+05   1.13e+04     17.767      0.000    1.79e+05    2.23e+05
decade_built_1940s              1.791e+05   8450.597     21.189      0.000    1.62e+05    1.96e+05
decade_built_1950s              1.196e+05   7873.767     15.191      0.000    1.04e+05    1.35e+05
decade_built_1960s              5.596e+04   7668.478      7.297      0.000    4.09e+04     7.1e+04
decade_built_1970s             -6975.1515   7695.416     -0.906      0.365   -2.21e+04    8108.647
decade_built_1980s             -3.352e+04   7532.831     -4.450      0.000   -4.83e+04   -1.88e+04
decade_built_1990s             -6.108e+04   7519.520     -8.122      0.000   -7.58e+04   -4.63e+04
decade_built_2000s             -3.398e+04   7050.643     -4.819      0.000   -4.78e+04   -2.02e+04
grade                           1.024e+05   2163.677     47.337      0.000    9.82e+04    1.07e+05
renovated_after_2005            4.985e+04    1.2e+04      4.146      0.000    2.63e+04    7.34e+04
==============================================================================
Omnibus:                    11578.294   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           432031.789
Skew:                           2.722   Prob(JB):                         0.00
Kurtosis:                      26.906   Cond. No.                     2.34e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.34e+06. This might indicate that there are
strong multicollinearity or other numerical proble

---

In [264]:
reg5_c = [
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
 'grade']

In [265]:
X = sm.add_constant(df.loc[:,reg5_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     4090.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:50:58   Log-Likelihood:            -2.3572e+05
No. Observations:               17249   AIC:                         4.715e+05
Df Residuals:                   17240   BIC:                         4.715e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           -6.74e+05   1.42e+04    -47.633      0.000   -7.02e+05   -6.46e+05
sqft_living                      140.3178      2.789     50.315      0.000     134.852     145.784
zip_mean_income                    2.7029      0.045     60.066      0.000       2.615       2.791
excess_bathrooms_cat_1_or_more   8.53e+04   8517.430     10.015      0.000    6.86e+04    1.02e+05
excess_bathrooms_cat_2_or_less  2.376e+04   4706.036      5.050      0.000    1.45e+04     3.3e+04
excess_bathrooms_cat_1_less      593.4821   3927.208      0.151      0.880   -7104.244    8291.208
view_cat_great_view             6.006e+05   1.32e+04     45.563      0.000    5.75e+05    6.26e+05
view_cat_ok_view                1.572e+05   5884.495     26.706      0.000    1.46e+05    1.69e+05
grade                           6.856e+04   2269.903     30.203      0.000    6.41e+04     7.3e+04
==============================================================================
Omnibus:                    11074.453   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           307237.674
Skew:                           2.647   Prob(JB):                         0.00
Kurtosis:                      22.986   Cond. No.                     1.29e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

---

In [261]:
reg6_c =[
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_-4.0',
 'excess_bathrooms_-3.0',
 'excess_bathrooms_-2.0',
 'excess_bathrooms_-1.0',
 'excess_bathrooms_1.0',
 'excess_bathrooms_2.0',
 'view_cat_great_view',
 'view_cat_ok_view',
'decade_built_adjusted_Pre-war',
 'decade_built_adjusted_1950s',
 'decade_built_adjusted_1960s',
 'decade_built_adjusted_1970s',
 'decade_built_adjusted_1980s',
 'decade_built_adjusted_1990s',
 'decade_built_adjusted_2000s',
 'grade',
'renovated_after_2005']

In [262]:
X =  sm.add_constant(df.loc[:,reg6_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     2204.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:50:47   Log-Likelihood:            -2.3426e+05
No. Observations:               17249   AIC:                         4.686e+05
Df Residuals:                   17229   BIC:                         4.687e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                          -7.81e+05   1.36e+04    -57.432      0.000   -8.08e+05   -7.54e+05
sqft_living                     143.4672      2.590     55.403      0.000     138.391     148.543
zip_mean_income                   2.6207      0.042     63.109      0.000       2.539       2.702
excess_bathrooms_-4.0         -2.758e+04   2.03e+04     -1.360      0.174   -6.73e+04    1.22e+04
excess_bathrooms_-3.0          -4.71e+04   8598.796     -5.477      0.000    -6.4e+04   -3.02e+04
excess_bathrooms_-2.0         -2.597e+04   4836.557     -5.369      0.000   -3.54e+04   -1.65e+04
excess_bathrooms_-1.0         -2.349e+04   3704.770     -6.341      0.000   -3.08e+04   -1.62e+04
excess_bathrooms_1.0           7.684e+04   8051.636      9.544      0.000    6.11e+04    9.26e+04
excess_bathrooms_2.0           2.183e+05   3.36e+04      6.499      0.000    1.52e+05    2.84e+05
view_cat_great_view            5.468e+05   1.22e+04     44.824      0.000    5.23e+05    5.71e+05
view_cat_ok_view               1.155e+05   5488.005     21.037      0.000    1.05e+05    1.26e+05
decade_built_adjusted_Pre-war  1.182e+05   5532.602     21.358      0.000    1.07e+05    1.29e+05
decade_built_adjusted_1950s    7247.7321   5914.532      1.225      0.220   -4345.351    1.88e+04
decade_built_adjusted_1960s   -5.308e+04   5763.611     -9.209      0.000   -6.44e+04   -4.18e+04
decade_built_adjusted_1970s   -1.113e+05   6015.660    -18.494      0.000   -1.23e+05   -9.95e+04
decade_built_adjusted_1980s    -1.31e+05   6036.337    -21.709      0.000   -1.43e+05   -1.19e+05
decade_built_adjusted_1990s   -1.539e+05   6192.577    -24.845      0.000   -1.66e+05   -1.42e+05
decade_built_adjusted_2000s   -1.259e+05   5624.503    -22.387      0.000   -1.37e+05   -1.15e+05
grade                          9.316e+04   2148.622     43.358      0.000    8.89e+04    9.74e+04
renovated_after_2005            6.92e+04   1.21e+04      5.698      0.000    4.54e+04     9.3e+04
==============================================================================
Omnibus:                    11532.161   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           419287.340
Skew:                           2.715   Prob(JB):                         0.00
Kurtosis:                      26.535   Cond. No.                     3.23e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.23e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [259]:
reg6_c =[
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
'decade_built_adjusted_Pre-war',
     'decade_built_adjusted_1940s',
 'decade_built_adjusted_1950s',
 'decade_built_adjusted_1960s',
 'decade_built_adjusted_1970s',
 'decade_built_adjusted_1980s',
 'decade_built_adjusted_1990s',
 'decade_built_adjusted_2000s',
'renovated_after_2005']

In [260]:
X = sm.add_constant(df.loc[:,reg6_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     2272.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:49:04   Log-Likelihood:            -2.3511e+05
No. Observations:               17249   AIC:                         4.703e+05
Df Residuals:                   17232   BIC:                         4.704e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                          -3.419e+05   9316.281    -36.704      0.000    -3.6e+05   -3.24e+05
sqft_living                      225.1694      2.035    110.644      0.000     221.180     229.158
zip_mean_income                    3.0464      0.042     71.892      0.000       2.963       3.130
excess_bathrooms_cat_1_or_more  1.079e+05   8264.605     13.056      0.000    9.17e+04    1.24e+05
excess_bathrooms_cat_2_or_less -7.146e+04   4913.894    -14.543      0.000   -8.11e+04   -6.18e+04
excess_bathrooms_cat_1_less    -4.617e+04   3904.694    -11.825      0.000   -5.38e+04   -3.85e+04
view_cat_great_view             5.675e+05   1.28e+04     44.246      0.000    5.42e+05    5.93e+05
view_cat_ok_view                1.252e+05   5783.533     21.651      0.000    1.14e+05    1.37e+05
decade_built_adjusted_Pre-war   1.609e+05   7908.284     20.340      0.000    1.45e+05    1.76e+05
decade_built_adjusted_1940s     9.842e+04   8803.863     11.179      0.000    8.12e+04    1.16e+05
decade_built_adjusted_1950s     6.574e+04   8287.035      7.933      0.000    4.95e+04     8.2e+04
decade_built_adjusted_1960s     5551.8631   8077.664      0.687      0.492   -1.03e+04    2.14e+04
decade_built_adjusted_1970s    -4.786e+04   8133.464     -5.884      0.000   -6.38e+04   -3.19e+04
decade_built_adjusted_1980s    -5.338e+04   7999.648     -6.672      0.000   -6.91e+04   -3.77e+04
decade_built_adjusted_1990s    -6.436e+04   7997.581     -8.048      0.000      -8e+04   -4.87e+04
decade_built_adjusted_2000s    -4.692e+04   7493.578     -6.261      0.000   -6.16e+04   -3.22e+04
renovated_after_2005            6.319e+04   1.28e+04      4.944      0.000    3.81e+04    8.82e+04
==============================================================================
Omnibus:                    10555.363   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           304061.229
Skew:                           2.453   Prob(JB):                         0.00
Kurtosis:                      22.975   Cond. No.                     1.88e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [198]:
grade_df = pd.get_dummies(df, columns = ['grade'], drop_first= True)

In [253]:
reg7_c =[
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
'decade_built_adjusted_Pre-war',
 'decade_built_adjusted_1940s',
 'decade_built_adjusted_1950s',
 'decade_built_adjusted_1960s',
 'decade_built_adjusted_1970s',
 'decade_built_adjusted_1980s',
 'decade_built_adjusted_1990s',
 'decade_built_adjusted_2000s',
'renovated_after_2005',
 'grade_cat_Low_Grade',
 'grade_cat_Mid_Grade']

In [255]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,year_most_recent_construction,years_since_most_recent_construction,yard,zip_total_households,zip_median_income,zip_mean_income,zip_total_families,zip_total_nonfamilies,log_price,log_sqft_living,10yr_ysmrc,renovated_after_2005,season_sold_Spring,season_sold_Summer,season_sold_Winter,condition_2,condition_3,condition_4,condition_5,condition_adjusted_3,condition_adjusted_4,condition_adjusted_5,grade_cat_Low_Grade,grade_cat_Mid_Grade,view_cat_great_view,view_cat_ok_view,excess_bathrooms_cat_1_less,excess_bathrooms_cat_1_or_more,excess_bathrooms_cat_2_or_less,excess_bathrooms_-4.0,excess_bathrooms_-3.0,excess_bathrooms_-2.0,excess_bathrooms_-1.0,excess_bathrooms_1.0,excess_bathrooms_2.0,decade_built_1900s,decade_built_1910s,decade_built_1920s,decade_built_1930s,decade_built_1940s,decade_built_1950s,decade_built_1960s,decade_built_1970s,decade_built_1980s,decade_built_1990s,decade_built_2000s,decade_built_adjusted_1940s,decade_built_adjusted_1950s,decade_built_adjusted_1960s,decade_built_adjusted_1970s,decade_built_adjusted_1980s,decade_built_adjusted_1990s,decade_built_adjusted_2000s,decade_built_adjusted_Pre-war
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,2014,10,1986,28,7858.0,15830,92225,107395,11422,4408,12.807653,7.635304,30.0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,2014,8,1977,37,2760.0,22922,111569,147430,12426,10496,13.670485,7.972466,40.0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,2014,8,1997,17,9008.0,13337,144247,186548,10531,2806,13.852806,8.234830,20.0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,2015,3,1990,25,12328.0,17646,101674,128063,11334,6312,14.214287,8.425078,20.0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,2014,7,2004,10,4101.0,28022,135287,161369,18806,9216,13.474428,7.843849,10.0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


In [256]:
X = sm.add_constant(df.loc[:,reg7_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.704
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     2282.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:48:11   Log-Likelihood:            -2.3438e+05
No. Observations:               17249   AIC:                         4.688e+05
Df Residuals:                   17230   BIC:                         4.689e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           3.391e+04   1.32e+04      2.577      0.010    8114.701    5.97e+04
sqft_living                      181.2917      2.263     80.103      0.000     176.856     185.728
zip_mean_income                    2.8215      0.041     68.721      0.000       2.741       2.902
excess_bathrooms_cat_1_or_more  8.938e+04   7938.268     11.259      0.000    7.38e+04    1.05e+05
excess_bathrooms_cat_2_or_less -5.867e+04   4724.788    -12.418      0.000   -6.79e+04   -4.94e+04
excess_bathrooms_cat_1_less    -3.675e+04   3752.430     -9.794      0.000   -4.41e+04   -2.94e+04
view_cat_great_view             5.361e+05   1.23e+04     43.490      0.000    5.12e+05     5.6e+05
view_cat_ok_view                1.186e+05   5548.318     21.375      0.000    1.08e+05    1.29e+05
decade_built_adjusted_Pre-war   1.613e+05   7656.872     21.072      0.000    1.46e+05    1.76e+05
decade_built_adjusted_1940s     1.006e+05   8607.020     11.690      0.000    8.37e+04    1.17e+05
decade_built_adjusted_1950s      5.94e+04   7957.806      7.465      0.000    4.38e+04     7.5e+04
decade_built_adjusted_1960s      697.7400   7745.454      0.090      0.928   -1.45e+04    1.59e+04
decade_built_adjusted_1970s    -4.742e+04   7797.960     -6.081      0.000   -6.27e+04   -3.21e+04
decade_built_adjusted_1980s    -6.737e+04   7678.248     -8.774      0.000   -8.24e+04   -5.23e+04
decade_built_adjusted_1990s    -8.855e+04   7693.994    -11.509      0.000   -1.04e+05   -7.35e+04
decade_built_adjusted_2000s     -5.41e+04   7186.985     -7.528      0.000   -6.82e+04      -4e+04
renovated_after_2005            7.247e+04   1.23e+04      5.910      0.000    4.84e+04    9.65e+04
grade_cat_Low_Grade            -3.347e+05   9403.902    -35.593      0.000   -3.53e+05   -3.16e+05
grade_cat_Mid_Grade            -2.663e+05   6966.671    -38.229      0.000    -2.8e+05   -2.53e+05
==============================================================================
Omnibus:                    10414.951   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           327300.281
Skew:                           2.375   Prob(JB):                         0.00
Kurtosis:                      23.805   Cond. No.                     1.95e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.95e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:

#grade test 

In [284]:
reg8_c =[
 'sqft_living',
 'zip_mean_income',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'excess_bathrooms_cat_1_less',
 'view_cat_great_view',
 'view_cat_ok_view',
'decade_built_adjusted_Pre-war',
 'decade_built_adjusted_1940s',
 'decade_built_adjusted_1950s',
 'decade_built_adjusted_1960s',
 'decade_built_adjusted_1970s',
 'decade_built_adjusted_1980s',
 'decade_built_adjusted_1990s',
 'decade_built_adjusted_2000s',
'renovated_after_2005',
'grade']

In [285]:
X = sm.add_constant(df.loc[:,reg8_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     2549.
Date:                Tue, 09 Mar 2021   Prob (F-statistic):               0.00
Time:                        19:59:14   Log-Likelihood:            -2.3405e+05
No. Observations:               17249   AIC:                         4.681e+05
Df Residuals:                   17231   BIC:                         4.683e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                          -9.449e+05   1.55e+04    -61.149      0.000   -9.75e+05   -9.15e+05
sqft_living                      145.7360      2.545     57.269      0.000     140.748     150.724
zip_mean_income                    2.5791      0.041     62.808      0.000       2.499       2.660
excess_bathrooms_cat_1_or_more  8.253e+04   7792.472     10.591      0.000    6.73e+04    9.78e+04
excess_bathrooms_cat_2_or_less -4.457e+04   4656.953     -9.570      0.000   -5.37e+04   -3.54e+04
excess_bathrooms_cat_1_less     -3.25e+04   3684.245     -8.821      0.000   -3.97e+04   -2.53e+04
view_cat_great_view             5.218e+05   1.21e+04     43.116      0.000    4.98e+05    5.46e+05
view_cat_ok_view                1.013e+05   5463.664     18.536      0.000    9.06e+04    1.12e+05
decade_built_adjusted_Pre-war   2.292e+05   7577.556     30.250      0.000    2.14e+05    2.44e+05
decade_built_adjusted_1940s     1.791e+05   8454.450     21.180      0.000    1.62e+05    1.96e+05
decade_built_adjusted_1950s     1.196e+05   7877.469     15.177      0.000    1.04e+05    1.35e+05
decade_built_adjusted_1960s     5.587e+04   7672.087      7.283      0.000    4.08e+04    7.09e+04
decade_built_adjusted_1970s    -7011.5960   7699.120     -0.911      0.362   -2.21e+04    8079.462
decade_built_adjusted_1980s     -3.35e+04   7536.496     -4.446      0.000   -4.83e+04   -1.87e+04
decade_built_adjusted_1990s    -6.104e+04   7523.194     -8.114      0.000   -7.58e+04   -4.63e+04
decade_built_adjusted_2000s    -3.396e+04   7054.088     -4.814      0.000   -4.78e+04   -2.01e+04
renovated_after_2005            5.093e+04    1.2e+04      4.235      0.000    2.74e+04    7.45e+04
grade                           1.025e+05   2163.402     47.375      0.000    9.83e+04    1.07e+05
==============================================================================
Omnibus:                    11557.170   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           428696.002
Skew:                           2.717   Prob(JB):                         0.00
Kurtosis:                      26.811   Cond. No.                     2.05e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.05e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [351]:
reg9_c = [
 'sqft_living',
 'waterfront',
 'zip_mean_income',
 'view_cat_great_view',
 'view_cat_ok_view',
 'excess_bathrooms_cat_1_less',
 'excess_bathrooms_cat_1_or_more',
 'excess_bathrooms_cat_2_or_less',
 'decade_built_adjusted_Pre-war',
 'decade_built_adjusted_1940s',
 'decade_built_adjusted_1950s',
 'decade_built_adjusted_1960s',
 'decade_built_adjusted_1970s',
 'decade_built_adjusted_1980s',
 'decade_built_adjusted_1990s',
 'decade_built_adjusted_2000s',
 'grade_cat_6',
 'grade_cat_7',
 'grade_cat_8',
 'grade_cat_9',
 'grade_cat_10',
 'grade_cat_11',
 'grade_cat_12',
 'grade_cat_13'
]

In [354]:
X = sm.add_constant(df.loc[:,reg9_c])
y = df['price']

model = sm.OLS(endog=y, exog=X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.745
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     2100.
Date:                Wed, 10 Mar 2021   Prob (F-statistic):               0.00
Time:                        09:39:09   Log-Likelihood:            -2.3310e+05
No. Observations:               17249   AIC:                         4.663e+05
Df Residuals:                   17224   BIC:                         4.664e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                          -3.192e+05   1.47e+04    -21.641      0.000   -3.48e+05    -2.9e+05
sqft_living                      131.0658      2.448     53.538      0.000     126.267     135.864
waterfront                      4.598e+05   1.92e+04     23.903      0.000    4.22e+05    4.97e+05
zip_mean_income                    2.6206      0.039     67.210      0.000       2.544       2.697
view_cat_great_view             3.007e+05   1.41e+04     21.395      0.000    2.73e+05    3.28e+05
view_cat_ok_view                9.787e+04   5185.776     18.872      0.000    8.77e+04    1.08e+05
excess_bathrooms_cat_1_less    -2.746e+04   3495.239     -7.856      0.000   -3.43e+04   -2.06e+04
excess_bathrooms_cat_1_or_more  5.786e+04   7408.251      7.810      0.000    4.33e+04    7.24e+04
excess_bathrooms_cat_2_or_less  -4.21e+04   4417.614     -9.531      0.000   -5.08e+04   -3.34e+04
decade_built_adjusted_Pre-war   1.908e+05   7242.733     26.340      0.000    1.77e+05    2.05e+05
decade_built_adjusted_1940s     1.322e+05   8141.070     16.242      0.000    1.16e+05    1.48e+05
decade_built_adjusted_1950s     8.803e+04   7525.532     11.697      0.000    7.33e+04    1.03e+05
decade_built_adjusted_1960s      3.19e+04   7339.009      4.347      0.000    1.75e+04    4.63e+04
decade_built_adjusted_1970s    -2.218e+04   7332.401     -3.024      0.002   -3.65e+04   -7804.526
decade_built_adjusted_1980s    -5.268e+04   7182.020     -7.335      0.000   -6.68e+04   -3.86e+04
decade_built_adjusted_1990s    -8.711e+04   7177.300    -12.137      0.000   -1.01e+05    -7.3e+04
decade_built_adjusted_2000s     -4.75e+04   6699.944     -7.089      0.000   -6.06e+04   -3.44e+04
grade_cat_6                     6.811e+04   1.31e+04      5.219      0.000    4.25e+04    9.37e+04
grade_cat_7                     1.455e+05   1.27e+04     11.440      0.000    1.21e+05     1.7e+05
grade_cat_8                       2.1e+05   1.32e+04     15.935      0.000    1.84e+05    2.36e+05
grade_cat_9                     3.137e+05   1.41e+04     22.257      0.000    2.86e+05    3.41e+05
grade_cat_10                    4.696e+05   1.55e+04     30.309      0.000    4.39e+05       5e+05
grade_cat_11                    7.186e+05   1.84e+04     38.960      0.000    6.82e+05    7.55e+05
grade_cat_12                    1.213e+06   2.79e+04     43.544      0.000    1.16e+06    1.27e+06
grade_cat_13                    1.913e+06   7.02e+04     27.253      0.000    1.78e+06    2.05e+06
==============================================================================
Omnibus:                     9081.329   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           217954.265
Skew:                           2.027   Prob(JB):                         